In [1]:
import cobra
from cobra.test import create_test_model

In [2]:
model = create_test_model() # if no argument is provided to this function, it loads a model for Salmonella enterica

In [3]:
model

Name,Salmonella_consensus_build_1
Memory address,0x011f66b898
Number of metabolites,1802
Number of reactions,2546
Number of groups,0
Objective expression,0.0 + 1.0*biomass_iRR1083_metals - 1.0*biomass_iRR1083_metals_reverse_00f08
Compartments,"Cytoplasm, Periplasm, Extracellular"


In [4]:
print(model.objective.expression)
print(model.objective.direction) # The objective function can be either minimized or maximized; this tells us what it's set as

0.0 + 1.0*biomass_iRR1083_metals - 1.0*biomass_iRR1083_metals_reverse_00f08
max


## Flux Balance Analysis
Without any additional changes to the model, the optimization problem is flux balance analysis.
If we optimize the model, we solve for the maximum obtainable flux through biomass given the current constraints.

In [5]:
model.optimize()

,fluxes,reduced_costs
12DGR120tipp,0.000000e+00,0.000000e+00
12DGR140tipp,0.000000e+00,8.498311e-13
12DGR141tipp,-6.057687e-19,0.000000e+00
12DGR160tipp,0.000000e+00,-2.677502e-11
12DGR161tipp,0.000000e+00,1.076727e-16
...,...,...
EX_guln__L_e,0.000000e+00,0.000000e+00
EX_udcpo5_e,0.000000e+00,0.000000e+00
EX_4hthr_e,0.000000e+00,0.000000e+00
EX_salchs2_e,0.000000e+00,0.000000e+00


You can explore the solution further by assigning the output of `model.optimize()` to a variable

In [6]:
solution = model.optimize()
solution.fluxes

12DGR120tipp    0.000000e+00
12DGR140tipp    0.000000e+00
12DGR141tipp   -6.057687e-19
12DGR160tipp    0.000000e+00
12DGR161tipp    0.000000e+00
                    ...     
EX_guln__L_e    0.000000e+00
EX_udcpo5_e     0.000000e+00
EX_4hthr_e      0.000000e+00
EX_salchs2_e    0.000000e+00
EX_tton_e       0.000000e+00
Name: fluxes, Length: 2546, dtype: float64

`solution.fluxes` is a pandas Series object (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html)
A Series is essentially a 1-D vector that has a label for each entry, called an "index" which is shown to the left of each
flux when we print them.

You can get the value for a specific entry in a Series by using the index names--both individual names, as well as a list of names, work.

In [7]:
print(solution.fluxes['12DGR120tipp'])
print(solution.fluxes[solution.fluxes.index[0:10]])

0.0
12DGR120tipp    0.000000e+00
12DGR140tipp    0.000000e+00
12DGR141tipp   -6.057687e-19
12DGR160tipp    0.000000e+00
12DGR161tipp    0.000000e+00
12DGR180tipp    0.000000e+00
12DGR181tipp    0.000000e+00
12PPDRtex       0.000000e+00
12PPDRtpp       0.000000e+00
12PPDStex       0.000000e+00
Name: fluxes, dtype: float64


The current media conditions for a model can be explored a few different ways in cobrapy.
The first way relies on knowing the reaction identifier prefix associated with exchange reactions.
To recap from our conversation, and exchange reaction is any reaction that moves a metabolite into/out of the boundary of the model,
e.g., it creates it from nothing.

Lets inspect the boundary reactions in the model. The `model.boundary` attribute points to all the reactions in the model that interact with the boundary.

In [8]:
model.boundary

[<Reaction DM_4HBA at 0x11fbb60b8>,
 <Reaction DM_5DRIB at 0x11fbb6320>,
 <Reaction DM_HMFURN at 0x11fbb6390>,
 <Reaction DM_OXAM at 0x11fbb6400>,
 <Reaction EX_chitob_e at 0x11fc4f080>,
 <Reaction EX_pydxn_e at 0x11fc4f278>,
 <Reaction EX_pydx_e at 0x11fc4f4e0>,
 <Reaction EX_12ppd__R_e at 0x11fc4f780>,
 <Reaction EX_12ppd__S_e at 0x11fc4f7f0>,
 <Reaction EX_14glucan_e at 0x11fc4f860>,
 <Reaction EX_15dap_e at 0x11fc4f8d0>,
 <Reaction EX_23camp_e at 0x11fc4f940>,
 <Reaction EX_23ccmp_e at 0x11fc4f9b0>,
 <Reaction EX_23cgmp_e at 0x11fc4fa20>,
 <Reaction EX_23cump_e at 0x11fc4fa90>,
 <Reaction EX_23dappa_e at 0x11fc4fb00>,
 <Reaction EX_26dap__M_e at 0x11fc4fb70>,
 <Reaction EX_2ddglcn_e at 0x11fc4fbe0>,
 <Reaction EX_34dhpac_e at 0x11fc4fc50>,
 <Reaction EX_3amp_e at 0x11fc4fcc0>,
 <Reaction EX_3cmp_e at 0x11fc4fd30>,
 <Reaction EX_3gmp_e at 0x11fc4fda0>,
 <Reaction EX_3hcinnm_e at 0x11fc4fe10>,
 <Reaction EX_3hpppn_e at 0x11fc4ffd0>,
 <Reaction EX_3ump_e at 0x11fc571d0>,
 <Reaction EX

As you can see, there are many boundary reactions. When cobra reaction objects are printed, the format is 
`<Reaction REACTION_ID at MEMORY_ADDRESS>`. You'll notice two reaction id prefixes in this list: EX and DM.
"EX_" is a very standard exchange reaction prefix across models. "DM_" also specifies an exchange reaction,
but it is a subtype called a demand reaction. Let's look at the difference between the two:

In [9]:
model.reactions.get_by_id('DM_4HBA')

Reaction identifier,DM_4HBA
Name,Sink needed to allow 4 hydroxy benzoate to leave system
Memory address,0x011fbb60b8
Stoichiometry,4hba_c --> 4-Hydroxy-benzyl-alcohol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
model.reactions.get_by_id('EX_pep_e')

Reaction identifier,EX_pep_e
Name,Phosphoenolpyruvate exchange
Memory address,0x01201d2ba8
Stoichiometry,pep_e --> Phosphoenolpyruvate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


They look identical, other than the metabolites involved, right? The lower and upper bounds are the same for both reaction types.
In practice, the only differences are that 1) a demand reaction often has a non-zero lower bound, which "forces" the model to produce
the metabolite involved, and 2) a demand reaction often involves a metabolite in the cytosol, not an external metabolite like most exchange reactions.
Here, the demand reaction doesn't "force" and flux because the lower bound is 0. But, it does allow the metabolite to leave the system without being transported out
of the cell, which is a useful model simplification when we don't know how a metabolite is transformed further or transported, but we know it's required somewhere in the network.
The terms "Demand" and "Sink" reaction are often used interchangably in the literature (e.g., the name of this demand reaction describes it as a "sink").

Based on these prefixes, we know the media composition is specifed by the exchange reactions which have "EX_" as the id prefix. Before we manipulate the media conditions, let's try the other method for obtaining these reactions, which uses the `model.medium` attribute.

In [11]:
model.medium

{'EX_ca2_e': 0.005,
 'EX_cit_e': 0.0005,
 'EX_cl_e': 5.016,
 'EX_co2_e': 18.5,
 'EX_cobalt2_e': 0.005,
 'EX_cu2_e': 0.005,
 'EX_fe3_e': 0.005,
 'EX_glyc_e': 41.0468020414812,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0,
 'EX_k_e': 6.0,
 'EX_mg2_e': 0.008,
 'EX_mn2_e': 0.005,
 'EX_mobd_e': 0.005,
 'EX_nh4_e': 15.0,
 'EX_o2_e': 18.5,
 'EX_pi_e': 0.337,
 'EX_so4_e': 1.0,
 'EX_thm_e': 2.96498354434133e-08,
 'EX_zn2_e': 0.005}

This is a much smaller list than all the reactions with the "EX_" prefix--this is because `model.medium` is a dictionary that maps reaction ids to the uptake rates for that reaction; if flux through an exchange reaction is disabled (e.g., it's removed from the media), the reaction is removed from the `model.medium` dictionary.

If we wanted to manipulate the media, we have to change the lower bound on the exchange reactions. Negative flux through an exchange reaction leads to a metabolite entering the environment (remember, they are defined as " met --> ", so negative flux creates the metabolite from nothing). The first way to do this is directly on the bounds of the exchange reactions:

In [12]:
# Get a reference to the reaction of interest
ex_rxn = model.reactions.get_by_id('EX_glyc_e')
ex_rxn

Reaction identifier,EX_glyc_e
Name,Glycerol exchange
Memory address,0x011fcb3518
Stoichiometry,glyc_e <=> Glycerol <=>
GPR,
Lower bound,-41.0468020414812
Upper bound,1000.0


In [13]:
print(ex_rxn.lower_bound)
print(ex_rxn.upper_bound)
print('Old bounds: ' + str(ex_rxn.bounds)) # an alternative way to access both the upper and lower bounds at once

# make the lower bound 0 to remove the metabolite from the media. Keep the upper bound at 1000 (the original value) so the metabolite can still be secreted by the cell if required for growth.
ex_rxn.bounds = (0, ex_rxn.upper_bound)

print('New bounds: ' + str(ex_rxn.bounds))

-41.0468020414812
1000.0
Old bounds: (-41.0468020414812, 1000.0)
New bounds: (0, 1000.0)


In [15]:
# The reaction we disabled is now removed from model.medium:
model.medium

{'EX_ca2_e': 0.005,
 'EX_cit_e': 0.0005,
 'EX_cl_e': 5.016,
 'EX_co2_e': 18.5,
 'EX_cobalt2_e': 0.005,
 'EX_cu2_e': 0.005,
 'EX_fe3_e': 0.005,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0,
 'EX_k_e': 6.0,
 'EX_mg2_e': 0.008,
 'EX_mn2_e': 0.005,
 'EX_mobd_e': 0.005,
 'EX_nh4_e': 15.0,
 'EX_o2_e': 18.5,
 'EX_pi_e': 0.337,
 'EX_so4_e': 1.0,
 'EX_thm_e': 2.96498354434133e-08,
 'EX_zn2_e': 0.005}

In [14]:
# Let's try to optimize for biomass production again
model.optimize()

cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'


<Solution infeasible at 0x11f6a4b38>

As you can see, we now get an infeasible solution when trying to optimize for flux through biomass, which means that there is no possible solution that can lead to production of biomass. The metabolite we removed was glycerol, which is a short carbon-containing metabolite. Let's see if we can restore biomass production by adding a different carbon source to the media--try glucose, which has an exchange reaction with the id 'EX_glc__D_e'.

In [17]:
model.reactions.get_by_id('EX_glc__D_e').bounds = (-10,1000)
model.optimize()

,fluxes,reduced_costs
12DGR120tipp,0.0,0.000000e+00
12DGR140tipp,0.0,-2.654174e-16
12DGR141tipp,0.0,-5.048710e-29
12DGR160tipp,0.0,4.339868e-16
12DGR161tipp,0.0,-4.291760e-16
...,...,...
EX_guln__L_e,0.0,-9.476345e-15
EX_udcpo5_e,0.0,0.000000e+00
EX_4hthr_e,0.0,0.000000e+00
EX_salchs2_e,0.0,0.000000e+00


Great, we restored biomass production. Let's use the `model.medium` attribute to reset the media conditions to the original glycerol-containing medium.

In [19]:
media = model.medium
media['EX_glyc_e'] = 41.046802041481 # add uptake of glycerol. This is an uptake rate, which is defined as a positive number. The lower bound will be this number multiplied by -1.
media['EX_glc__D_e'] = 0 # remove glucose
model.medium = media

Important note: you cannot set the uptake rates directly in the media dictionary by using it without assigning it to a new variable; e.g. this will not work:
`model.medium['EX_glc__D_e'] = 0`. The medium has to be extracted/copied from the model, manipulated, and then set as the new media condition using `model.medium = media`.
This is because of the getter/setter architecture of the `model.medium` attribute, a common approach in python that can make code much simpler to maintain in some circumstances.

In [23]:
new_solution = model.optimize()
new_solution

,fluxes,reduced_costs
12DGR120tipp,0.0,0.000000e+00
12DGR140tipp,0.0,-1.449815e-15
12DGR141tipp,0.0,0.000000e+00
12DGR160tipp,0.0,5.297457e-16
12DGR161tipp,0.0,1.896567e-11
...,...,...
EX_guln__L_e,0.0,-2.046608e-14
EX_udcpo5_e,0.0,0.000000e+00
EX_4hthr_e,0.0,0.000000e+00
EX_salchs2_e,0.0,0.000000e+00


In [22]:
# Check out the flux through the glycerol exchange reaction in the new solution to verify it's being used
new_solution.fluxes['EX_glyc_e']

-9.455426006534676

## Flux Variability Analysis
FBA is great for determining optimal growth rates, but there are nearly always an infinite number of solutions (sets of flux values for each reaction in the network) that lead to the same maximal value for the objective (here, flux through the biomass reaction). Flux Variability Analysis (FVA) let's us identify the minimal and maximal flux through individual reactions in the network that still allow an objective value above a specified threshold. In other words, FVA tells us the minimum and maximum flux through a single reaction that allow at least a specified amount of flux through biomass.

We can use FVA to identify components of the media that must be consumed in order for the model to grow. Let's get the exchange reactions that are in `model.medium` and perform FVA for each reaction. The critical arguments for the FVA function are the model FVA is being performed on, the list of reactions that FVA will be performed on (the process is performed separately for each reaction in the list), and the fraction of the optimum value of the objective that needs to be maintained when manipulating flux through the target reaction (where the target reaction is the reaction being minimized/maximized in FVA). Here, we'll use `fraction_of_optimum = 1.0` to require 100% of the optimum value to be maintained during FVA.

In [28]:
from cobra.flux_analysis import flux_variability_analysis
# the keys in the medium dictionary are the reaction ids for each exchange reaction. We also have to convert it to a list to work with the FVA function
media_exchange_reactions = list(model.medium.keys()) 
print(media_exchange_reactions)
fva_results = flux_variability_analysis(model,media_exchange_reactions,fraction_of_optimum=1.0)
fva_results

['EX_ca2_e', 'EX_cit_e', 'EX_cl_e', 'EX_co2_e', 'EX_cobalt2_e', 'EX_cu2_e', 'EX_fe3_e', 'EX_glyc_e', 'EX_h2o_e', 'EX_h_e', 'EX_k_e', 'EX_mg2_e', 'EX_mn2_e', 'EX_mobd_e', 'EX_nh4_e', 'EX_o2_e', 'EX_pi_e', 'EX_so4_e', 'EX_thm_e', 'EX_zn2_e']


,minimum,maximum
EX_ca2_e,-1.800098e-03,-0.001800
EX_cit_e,-5.000000e-04,0.000000
EX_cl_e,-1.800098e-03,-0.001800
EX_co2_e,-1.850000e+01,36.002003
EX_cobalt2_e,-1.200065e-03,-0.001200
EX_cu2_e,-1.200065e-03,-0.001200
EX_fe3_e,-5.000000e-03,-0.002785
EX_glyc_e,-4.104680e+01,-7.020229
EX_h2o_e,1.365091e+01,116.454065
EX_h_e,1.262146e+00,83.886574


The FVA function returns a pandas dataframe, which is like a multi-dimensional Series. Indexing works the same way--each row can have a name, called the index, and each column can have a name as well. Columns are accessed by their names, but rows have to be accessed with the `.loc` operator to access by their name. (See this extremely thorough tutorial, which I've googled a million times and still use for reference frequently: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html)

We'll use some logical operations to extract metabolites for which uptake was required for optimal growth. Remember that uptake into the cell for an exchange reaction means that there was negative flux. If the maximum flux value obtained from FVA is negative, the metabolite was taken up by the model.

In [32]:
required_uptake = fva_results.loc[fva_results['maximum'] < 0]
required_uptake

,minimum,maximum
EX_ca2_e,-0.001800,-0.001800
EX_cl_e,-0.001800,-0.001800
EX_cobalt2_e,-0.001200,-0.001200
EX_cu2_e,-0.001200,-0.001200
EX_fe3_e,-0.005000,-0.002785
EX_glyc_e,-41.046802,-7.020229
EX_k_e,-0.067489,-0.067489
EX_mg2_e,-0.003000,-0.003000
EX_mn2_e,-0.001200,-0.001200
EX_mobd_e,-0.001200,-0.001200


The list of reactions is nice, but what if we want to extract the actual metabolites that are involved, in an automated way?

In [39]:
list_of_metabolites = []
for ex_rxn in required_uptake.index:
    rxn_obj = model.reactions.get_by_id(ex_rxn)
    met = rxn_obj.reactants[0] # in exchange reactions, there is only 1 metabolite, and it is always a reactant rather than a product
    list_of_metabolites.append(met.name) # get the name of the metabolite; we could also extract the entire metabolite object, or the metabolite.id

print(list_of_metabolites)

['Calcium', 'Chloride', 'Co2', 'Cu2', 'Fe3', 'Glycerol', 'potassium', 'magnesium', 'Mn2', 'Molybdate-MoO4', 'Ammonium', 'O2', 'Phosphate', 'Sulfate', 'Zinc']


we can do the same thing in the one-liner below, which takes advantage of some python tricks. This can shorten your code, but requires understanding how
this trick works (Here's an intro to this specific trick, known as list comprehension: https://www.pythonforbeginners.com/basics/list-comprehensions-in-python and also a lot of detail on how this is allowed in python and the benefits: https://wiki.python.org/moin/Generators)

In [35]:
print([model.reactions.get_by_id(ex_rxn).reactants[0].name for ex_rxn in required_uptake.index])

['ca2_e',
 'cl_e',
 'cobalt2_e',
 'cu2_e',
 'fe3_e',
 'glyc_e',
 'k_e',
 'mg2_e',
 'mn2_e',
 'mobd_e',
 'nh4_e',
 'o2_e',
 'pi_e',
 'so4_e',
 'zn2_e']